# Decentralized Machine Learning

In [41]:
import warnings
warnings.simplefilter('ignore')
# warnings.filterwarnings('ignore')

### Client Setup

In [61]:
from web3 import Web3, HTTPProvider


web3 = Web3(HTTPProvider('http://localhost:8545'))
# web3 = Web3(web3.currentProvider)

PASSPHRASE = 'panda'
TEST_ACCOUNT = web3.eth.coinbase

acct = web3.personal.newAccount(PASSPHRASE)
print("New Account:", acct)
print("Balance:", web3.eth.getBalance(acct))
web3.eth.sendTransaction({"from": TEST_ACCOUNT, "to": acct, "value": 9999999999})
print("Balance:", web3.eth.getBalance(acct))
# print("Accounts on the Network:", web3.eth.accounts)
# print("Personal Accounts:", web3.personal.listAccounts)

New Account: 0x3229616ebb8f78D6a42F5395A55D9F53C59b3965
Balance: 0
Balance: 9999999999


### Contract Deployment

In [66]:
from solc import compile_source
import pprint


# CONTRACT_NAME = 'Greeter.sol'
CONTRACT_NAME = '../contracts/Query.sol'
ARG1 = 3
ARG2 = [0, 1, 2]

def compile_source_file(file_path):
   with open(file_path, 'r') as f:
      source = f.read()

   return compile_source(source)


def deploy_contract(w3, contract_interface, arg):
    tx_hash = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']).constructor(ARG1, ARG2).transact({"from": acct})

    address = w3.eth.getTransactionReceipt(tx_hash)['contractAddress']
    return address


def wait_for_receipt(w3, tx_hash, poll_interval):
   while True:
       tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
       if tx_receipt:
         return tx_receipt
       time.sleep(poll_interval)
    
contract_source_path = CONTRACT_NAME
compiled_sol = compile_source_file(contract_source_path)

pp = pprint.PrettyPrinter()

contract_id, contract_interface = compiled_sol.popitem()
# pp.pprint(contract_interface['abi'])

# print(web3.eth.accounts)
# print(acct)
web3.personal.unlockAccount(acct, PASSPHRASE)

address = deploy_contract(web3, contract_interface, VECLEN)
print("Deployed {0} to: {1}\n".format(contract_id.replace('<stdin>:', ''), address))

Deployed Query to: 0x4651c03987103098EA9765444B0B582B6623CDEf



### Event Listening

In [67]:
def txn_digest(txn):
    return "Contract address: {0}\nEvent: {1}\nArg: {2}".format(
        txn['address'], txn['event'], txn['args'])

def event_callback(arg):
    print(txn_digest(arg))

contract_obj = web3.eth.contract(
   address=address,
   abi=contract_interface['abi'])

# pp.pprint(contract_obj.events)

# tx_retval = contract_obj.functions.sendResponse([0, 1, 2], 1, 3).call({'from': TEST_ACCOUNT})
tx_hash = contract_obj.functions.sendResponse([0, 1, 2], 0, 3).transact({'from': TEST_ACCOUNT})
tx_receipt = web3.eth.getTransactionReceipt(tx_hash)
# print(tx_receipt)

ResponseReceived_event = contract_obj.events.ResponseReceived()

ResponseReceived_event.createFilter(fromBlock='latest')
# ResponseReceived_event.get_all_entries()
# client_selected_event.createFilter({'filter': {'arg1':10}})

# contract_obj.functions.keyList().call({'from': TEST_ACCOUNT})

log1 = contract_obj.events.ResponseReceived().processReceipt(tx_receipt)
# log2 = contract_obj.events.ResponseReceived().processReceipt(tx_receipt)

print(log1[0])
event_callback(log1[0])

contract_obj.functions.sendResponse([0, 1, 2], 1, 3).transact({'from': TEST_ACCOUNT})
contract_obj.functions.sendResponse([0, 1, 2], 2, 3).transact({'from': TEST_ACCOUNT})
contract_obj.functions.inverseScale().transact({'from': TEST_ACCOUNT})

AttributeDict({'args': AttributeDict({'n': 1}), 'event': 'ResponseReceived', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0xfc60f7dc6775103bee477cc5471385f182d2557300062036960a234434d802c2'), 'address': '0x4651c03987103098EA9765444B0B582B6623CDEf', 'blockHash': HexBytes('0x8f2eddfa5b390c442467842e759405e19fb5fe2fdc389c12c2c5ef04a86aaa4a'), 'blockNumber': 35})
Contract address: 0x4651c03987103098EA9765444B0B582B6623CDEf
Event: ResponseReceived
Arg: AttributeDict({'n': 1})


ValueError: {'message': 'VM Exception while processing transaction: invalid opcode', 'code': -32000, 'data': {'stack': 'c: VM Exception while processing transaction: invalid opcode\n    at Function.c.fromResults (/usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:313802)\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:537769\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:2006464\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:89955\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:75462\n    at o (/usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:78562)\n    at f (/usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:78484)\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:78286\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:89935\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:656606\n    at l (/usr/local/lib/node_modules/ga